In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import gc
import PIL
import cv2
import uuid
import shutil
import random
import glob as gb
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from tqdm import tqdm  # Progress bar
from scipy.special import gamma

import keras
from keras.optimizers import *
from keras.regularizers import l1_l2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler
from keras.layers import Conv2D, MaxPool2D, BatchNormalization

from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Function to get the version of a module if it exists
def get_version(module):
    try:
        return module.__version__
    except AttributeError:
        return "Version info not available"

# Print versions of each module
print("os:", os.__version__ if hasattr(os, '__version__') else "OS version not available")
print("gc:", get_version(gc))
print("PIL:", get_version(PIL))
print("cv2:", cv2.__version__)
print("uuid:", get_version(uuid))
print("shutil:", get_version(shutil))
print("random:", get_version(random))
print("glob:", get_version(gb))
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("tensorflow:", tf.__version__)
import matplotlib
print("matplotlib:", matplotlib.__version__)
import tqdm
print("tqdm:", tqdm.__version__)
import scipy
print("scipy:", scipy.__version__)
print("keras:", keras.__version__)
print("tensorflow:", tf.__version__)
import sklearn
print("sklearn:", sklearn.__version__)  # sklearn version

In [ ]:
path = "/kaggle/input/cbis-ddsm-breast-cancer-image-dataset"

for files in os.listdir(path):
    files_dir = os.path.join(path, files)


    if files == 'jpeg':   # to pass 6774 files 
        pass
    else:
        for file in os.listdir(files_dir):
            print(file)

In [ ]:
dicom_df = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/dicom_info.csv")

In [ ]:
dicom_df.head()

In [ ]:
dicom_df.describe().T

In [ ]:
dicom_df.info()

In [ ]:
dicom_df.SeriesDescription.unique()

In [ ]:
dicom_df.SeriesDescription.value_counts()

In [ ]:
cropped_images = dicom_df[dicom_df.SeriesDescription=="cropped images"].image_path
cropped_images.head()

In [ ]:
full_mammogram = dicom_df[dicom_df.SeriesDescription=="full mammogram images"].image_path
full_mammogram.head()

In [ ]:
roi_mask = dicom_df[dicom_df.SeriesDescription=="ROI mask images"].image_path
roi_mask.head()

In [ ]:
# # delete dicom_df after finished use it
del dicom_df;    gc.collect()

In [ ]:
def replace_path(sample, old_path, new_path):
    return sample.replace(old_path, new_path, regex=True)

In [ ]:
def plot_smaples(sample, row=15, col=15):
    plt.figure(figsize=(row, col))
    for i, file in enumerate(sample[0:5]):
        cropped_images_show = PIL.Image.open(file)
        gray_img= cropped_images_show.convert("L")
        plt.subplot(1,5,i+1)
        plt.imshow(gray_img, cmap='gray')
        plt.axis('off')
    plt.show()

In [ ]:
correct_dir = "../input/cbis-ddsm-breast-cancer-image-dataset/jpeg"

In [ ]:
cropped_images = replace_path(cropped_images, "CBIS-DDSM/jpeg", correct_dir)
print('Cropped Images paths:')
print(cropped_images.iloc[0])

In [ ]:
plot_smaples(cropped_images, 15, 15)

In [ ]:
full_mammogram = replace_path(full_mammogram, "CBIS-DDSM/jpeg", correct_dir)
print('\nFull mammo Images paths:')
print(full_mammogram.iloc[0])

In [ ]:
plot_smaples(full_mammogram)

In [ ]:
roi_mask = replace_path(roi_mask, "CBIS-DDSM/jpeg", correct_dir)
print('\nROI Mask Images paths:')
print(roi_mask.iloc[0])

In [ ]:
plot_smaples(roi_mask)

In [ ]:
def get_image_file_name(data, new_dict):
    """
    /kaggle
    /input
    /cbis-ddsm-breast-cancer-image-dataset
    /jpeg
    /1.3.6.1.4.1.9590.100.1.2.129308726812851964007517874181459556304 [5]
    /1-172.jpg
    
    return path at index [5] after split depends on split('\')
    """
    for dicom in data:
        key = dicom.split('/')[4]
#         print(key)
        new_dict[key] = dicom
    print(f"the length of dataset ==> {len(new_dict.keys())}")

In [ ]:
cropped_images_dict = dict()
full_mammo_dict = dict()
roi_img_dict = dict()

get_image_file_name(cropped_images, cropped_images_dict)
get_image_file_name(full_mammogram, full_mammo_dict)
get_image_file_name(roi_mask, roi_img_dict)

In [ ]:
next(iter((cropped_images.items())))

In [ ]:
next(iter((cropped_images_dict.items())))
print()
print(len(cropped_images_dict.keys()))

In [ ]:
next(iter((full_mammo_dict.items())))
print()
print(len(full_mammo_dict.keys()))

In [ ]:
next(iter((roi_img_dict.items())))
print()
print(len(roi_img_dict.keys()))

In [ ]:
del cropped_images, full_mammogram, roi_mask;    gc.collect()

In [ ]:
# fix image paths
def fix_image_path(data):
    """Correct dicom paths to correct image paths."""
    for indx, image in enumerate(data.values):
#         print(f"Image Path: {image[11]}")

        img_name = image[11].split('/')[2]
#         print(f"Looking for key: {img_name}")  # Debugging step

        if img_name in full_mammo_dict:
            data.iloc[indx, 11] = full_mammo_dict[img_name]
        else:
            data.iloc[indx, 11] = None
#             print(f"KeyError: '{img_name}' not found in full_mammo_dict")  # Debugging step
        
        img_name = image[12].split('/')[2]
        if img_name in cropped_images_dict:
            data.iloc[indx, 12] = cropped_images_dict[img_name]
        else:
            data.iloc[indx, 11] = None
            # print(f"KeyError: '{img_name}' not found in cropped_images_dict")  # Debugging step

        img_name = image[13].split('/')[2]
        if img_name in roi_img_dict:
            data.iloc[indx, 13] = roi_img_dict[img_name]

        else:
            data.iloc[indx, 13] = None

In [ ]:
mass_train = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_train_set.csv")
mass_test  = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_test_set.csv")

In [ ]:
mass_train.head()

In [ ]:
mass_train.iloc[:, 11].head()

In [ ]:
fix_image_path(mass_train)

In [ ]:
mass_train.info()

In [ ]:
mass_train = mass_train.rename(columns={'left or right breast': 'left_or_right_breast',
                                        'image view': 'image_view',
                                        'abnormality id': 'abnormality_id',
                                        'abnormality type': 'abnormality_type',
                                        'mass shape': 'mass_shape',
                                        'mass margins': 'mass_margins',
                                        'image file path': 'image_file_path',
                                        'cropped image file path': 'cropped_image_file_path',
                                        'ROI mask file path': 'ROI_mask_file_path'})
mass_train.head(5)

In [ ]:
mass_train.pathology.unique()

In [ ]:
mass_test.head()

In [ ]:
mass_train.iloc[:, 11].head()

In [ ]:
fix_image_path(mass_test)

In [ ]:
mass_test = mass_test.rename(columns={'left or right breast': 'left_or_right_breast',
                                      'image view': 'image_view',
                                      'abnormality id': 'abnormality_id',
                                      'abnormality type': 'abnormality_type',
                                      'mass shape': 'mass_shape',
                                      'mass margins': 'mass_margins',
                                      'image file path': 'image_file_path',
                                      'cropped image file path': 'cropped_image_file_path',
                                      'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
mass_test.head()

In [ ]:
print(f'Shape of mass_train: {mass_train.shape}')
print(f'Shape of mass_test: {mass_test.shape}')

In [ ]:
calc_train = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_train_set.csv")
calc_test  = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_test_set.csv")

In [ ]:
calc_train.head()

In [ ]:
calc_train.iloc[:,11].head()

In [ ]:
calc_train = calc_train.rename(columns={'left or right breast': 'left_or_right_breast',
                                        'image view': 'image_view',
                                        'abnormality id': 'abnormality_id',
                                        'abnormality type': 'abnormality_type',
                                        'mass shape': 'mass_shape',
                                        'mass margins': 'mass_margins',
                                        'image file path': 'image_file_path',
                                        'cropped image file path': 'cropped_image_file_path',
                                        'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
calc_train.head()

In [ ]:
fix_image_path(calc_train)

In [ ]:
calc_test.head()

In [ ]:
calc_test.iloc[:,11].head()

In [ ]:
calc_test = calc_test.rename(columns={'left or right breast': 'left_or_right_breast',
                                      'image view': 'image_view',
                                      'abnormality id': 'abnormality_id',
                                      'abnormality type': 'abnormality_type',
                                      'mass shape': 'mass_shape',
                                      'mass margins': 'mass_margins',
                                      'image file path': 'image_file_path',
                                      'cropped image file path': 'cropped_image_file_path',
                                      'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
calc_test.head()

In [ ]:
fix_image_path(calc_test)

In [ ]:
print(f'Shape of mass_train: {calc_train.shape}')
print(f'Shape of mass_test: {calc_test.shape}')

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np

def display_images(dataset, column, number):
    """Displays images in dataset, handling missing files and converting formats."""
    
    # create figure and axes
    fig, axes = plt.subplots(1, number, figsize=(15, 5))
    
    # Loop through rows and display images
    for index, (i, row) in enumerate(dataset.head(number).iterrows()):
        image_path = row[column]
        
       # Check if image_path is valid (not None) and exists
        if image_path is None or not os.path.exists(image_path):
            # print(f"File not found or invalid path: {image_path}")
            continue
        
        image = cv2.imread(image_path)
        
        # Handle case when image can't be read
        if image is None:
            # print(f"Error reading image: {image_path}")
            continue
        
        # Convert BGR to RGB if needed (for correct color display)
        if len(image.shape) == 3 and image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        ax = axes[index]
        ax.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
        ax.set_title(f"{row['pathology']}")
        ax.axis('off')
        print(np.array(image).shape)
    
    plt.tight_layout()
    plt.show()


In [ ]:
print('Full Mammograms:\n')
display_images(mass_train, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(mass_train, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(mass_train, 'ROI_mask_file_path', 5)

In [ ]:
print('Full Mammograms:\n')
display_images(mass_test, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(mass_test, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(mass_test, 'ROI_mask_file_path', 5)

In [ ]:
print('Full Mammograms:\n')
display_images(calc_train, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(calc_train, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(calc_train, 'ROI_mask_file_path', 5)

In [ ]:
print('Full Mammograms:\n')
display_images(calc_test, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(calc_test, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(calc_test, 'ROI_mask_file_path', 5)

In [ ]:
full_dataset = pd.concat([calc_train, calc_test], axis=0)

In [ ]:
del mass_train, mass_test, calc_train, calc_test;    gc.collect()

In [ ]:
class_mapper = {'MALIGNANT': 1, 'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 0} 

In [ ]:
target_size = (224, 224, 3)

In [ ]:
full_dataset['labels'] = full_dataset['pathology'].replace(class_mapper).infer_objects(copy=False)

full_images = np.array(full_dataset[full_dataset["image_file_path"].notna()]["image_file_path"].tolist())
full_labels = np.array(full_dataset[full_dataset["image_file_path"].notna()]["labels"].tolist())

In [ ]:
len(full_images)

In [ ]:
full_labels_series = pd.Series(full_labels)

# Count the occurrences of each class
label_counts = full_labels_series.value_counts()

# Assuming 0 = benign and 1 = malignant
benign_count = label_counts.get(0, 0)
malignant_count = label_counts.get(1, 0)

print(f"Benign images: {benign_count}")
print(f"Malignant images: {malignant_count}")

In [ ]:
num_classes = len(full_dataset['labels'].unique())
num_classes

In [ ]:
class_names = ['Benign', 'Malignant']

In [ ]:
dir_path = '/kaggle/working/'

# # Loop through all files and subdirectories in the working directory
for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove the file or symbolic link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove the directory and its contents
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print("All files and subdirectories in '/kaggle/working/' have been removed.")

In [ ]:
def images_count():
    zero_class_count = len(os.listdir("/kaggle/working/merged_images/0"))
    one_class_count  = len(os.listdir("/kaggle/working/merged_images/1"))

    print(f"Number of images in class 0: {zero_class_count}")
    print(f"Number of images in class 1: {one_class_count}")

In [ ]:
def augment_image(image):
    # Apply data augmentation using tf.image functions
    image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.3)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    return image

# Function to resize image to (224, 224, 3)
def resize_image(image_tensor):
    return tf.image.resize(image_tensor, [224, 224])

# Function to balance classes by augmenting images
def copy_images_with_unique_filenames(images, labels, source, destination, target_count=None):
    """
    Copy images from source to destination in subfolders '0' and '1',
    ensuring unique filenames and applying data augmentation and balancing.
    """
    benign_images = 0
    malignant_images = 0
    skipped_images = []

    # Create the destination subfolders '0' and '1'
    category_dest_dir_zero = os.path.join(destination, '0')
    os.makedirs(category_dest_dir_zero, exist_ok=True)

    category_dest_dir_one = os.path.join(destination, '1')
    os.makedirs(category_dest_dir_one, exist_ok=True)

    benign_images_list = []
    malignant_images_list = []

    for i, (image, label) in enumerate(zip(images, labels)):
#         img_name = data_frame.REFNUM[i]
#         abs_path = os.path.join(source, img_name + '.pgm')

        if os.path.exists(image):
            try:
                # Generate a unique filename
                filename = os.path.basename(image)
                unique_filename = f"{uuid.uuid4().hex}_{filename}"
        
                # Open the image using PIL
                with Image.open(image) as img:
                    # Convert the image to RGB mode (for saving as JPEG)
                    img = img.convert('RGB')
                    # Augment the image (convert it to a Tensor first)
                    img_tensor = tf.convert_to_tensor(img)
                    # Resize the image to (224, 224, 3)
                    resized_img_tensor = resize_image(img_tensor)
                    augmented_image_tensor = augment_image(resized_img_tensor)
                    # Convert Tensor back to PIL image for saving
                    augmented_image = tf.keras.preprocessing.image.array_to_img(augmented_image_tensor)

                    if label == 0:
                        benign_images_list.append(unique_filename)
                        dest_path = os.path.join(category_dest_dir_zero, unique_filename)
#                         augmented_image.save(dest_path, 'JPEG')
                        augmented_image.save(dest_path, 'JPEG')
                        benign_images += 1

                    elif label == 1:
                        malignant_images_list.append(unique_filename)
                        dest_path = os.path.join(category_dest_dir_one, unique_filename)
#                         augmented_image.save(dest_path, 'JPEG')
                        augmented_image.save(dest_path, 'JPEG')
                        malignant_images += 1
                        
#                 del img, img_tensor, resized_img_tensor, augmented_image_tensor, augmented_image
#                 gc.collect()
            except Exception as e:
                print(f"Error copying image {image}: {e}")
                skipped_images.append(image)
        else:
            print(f"Image not found: {image}")
            skipped_images.append(image)

    # If balancing is needed, duplicate/augment images from the smaller class
    benign_count = len(benign_images_list)
    malignant_count = len(malignant_images_list)

    if benign_count < malignant_count:
#         augment_and_save_images(benign_images_list, category_dest_dir_zero, target_count - benign_count)
        augment_and_save_images(benign_images_list, category_dest_dir_zero, malignant_count - benign_count)

    elif malignant_count < benign_count:
        augment_and_save_images(malignant_images_list, category_dest_dir_one, benign_count - malignant_count)
    print("data balancing")
    images_count()
    augment_and_save_images(benign_images_list, category_dest_dir_zero, target_count)
    augment_and_save_images(malignant_images_list, category_dest_dir_one, target_count)
    print("data augmentation")
    images_count()
    
    print(f"\nCopying complete.")
    print(f"Benign images copied (label 0): {benign_images}")
    print(f"Benign count (label 0): {benign_count}")
    print(f"Malignant images copied (label 1): {malignant_images}")
    print(f"Malignant count (label 1): {malignant_count}")
    print(f"Total skipped images: {len(skipped_images)}")
    if skipped_images:
        print("Skipped images:")
        for img in skipped_images:
            print(img)
            
    del skipped_images, benign_images_list, malignant_images_list
    gc.collect()

# Function to augment and save images to balance the dataset
def augment_and_save_images(images_list, destination_dir, num_augments):
    """
    Augment and save images to balance the dataset.
    """
    for i in range(num_augments):
        img_name = random.choice(images_list)
        abs_path = os.path.join(destination_dir, img_name)

        try:
            with Image.open(abs_path) as img:
                img = img.convert('RGB')
                # Augment the image
                img_tensor = tf.convert_to_tensor(img)
                # Resize the image
#                 resized_img_tensor = resize_image(img_tensor)
                augmented_image_tensor = augment_image(img_tensor)
                # Convert Tensor back to PIL image for saving
                augmented_image = tf.keras.preprocessing.image.array_to_img(augmented_image_tensor)
                # Remove the original extension from img_name 1-285.jpg --> 1-285
                img_name_without_ext = os.path.splitext(img_name)[0]
                # Save augmented image with a unique name
                augmented_image.save(os.path.join(destination_dir, img_name_without_ext + f'_aug{i}.jpg'), 'JPEG')
            
#                 del img, img_tensor, augmented_image_tensor, augmented_image, img_name_without_ext
#                 gc.collect()
        except Exception as e:
            print(f"Error augmenting image {abs_path}: {e}")

# Example usage
source_dir = "/kaggle/input/mias-mammography/all-mias"
destination_dir = "/kaggle/working/merged_images"

# target_count=0 meaning no Augmentation, There's just Data-Balance
target_count = int((len(full_labels) * 5) // 2)
copy_images_with_unique_filenames(full_images, full_labels, source_dir, destination_dir, target_count)

In [ ]:
target_count

In [ ]:
images_count()

In [ ]:
import tensorflow as tf
data_dir = '/kaggle/working/merged_images'  # Update with your dataset path

# Create a dataset for the entire data to use for split
full_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    # image_size=(224, 224),
    image_size=(224, 224),
    seed=30,
    shuffle=True,
    batch_size=13  # Set your desired batch size
)
# Calculate the total number of samples
total_samples = tf.data.experimental.cardinality(full_dataset).numpy()

# Step 3: Split the dataset into train, validation, and test sets
train_size = int(0.7 * total_samples)                 # 70% for training
# val_size   = int(0.15 * total_samples)                # 20% for validation
# test_size = total_samples - train_size - val_size     # 10% for testing
test_size = total_samples - train_size                # 30% for testing

# Create train, validation, and test datasets
train_dataset       = full_dataset.take(train_size)
# validation_dataset  = full_dataset.skip(train_size).take(val_size)
# test_dataset        = full_dataset.skip(train_size + val_size)
test_dataset        = full_dataset.skip(train_size)

# Step 4: Apply Augmentation to the Training Dataset
# train_dataset = train_dataset.map(
#     lambda x, y: (data_augmentation(x, training=True), y),
#     num_parallel_calls=tf.data.experimental.AUTOTUNE
# )

# Step 5: Prefetch to Improve Performance
train_dataset      = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
# validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset       = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Print the number of samples in each dataset
print(f"Train samples:      {train_size}     batches(8) ==> {train_size*13}")
# print(f"Validation samples: {val_size}       batches(13) ==> {val_size*13}")
print(f"Test samples:       {test_size}      batches(8) ==> {test_size*13}")

# del full_dataset, total_samples, train_size, val_size, test_size;    gc.collect()
del full_dataset, total_samples, train_size, test_size;    gc.collect()


In [ ]:
from tensorflow.keras.applications import (ResNet50,
                                           Xception,
                                           VGG19,
                                           InceptionV3,
                                           DenseNet121,
                                           NASNetMobile,
                                           MobileNetV2,
                                           MobileNet,
                                           EfficientNetV2B0,
                                           EfficientNetV2S,
                                           EfficientNetV2L,
                                           ConvNeXtBase)

def create_model(model_name, trainable_layers, dropout_value, save_weights=False):
    model_dict = {
        "ResNet50":         ResNet50,
        "Xception":         Xception,
        "VGG19":         VGG19,
        "InceptionV3":      InceptionV3,
        "DenseNet121":      DenseNet121,
        "NASNetMobile":     NASNetMobile,
        "MobileNetV2":      MobileNetV2,
        "MobileNet":        MobileNet,
        "EfficientNetV2B0": EfficientNetV2B0,
        "EfficientNetV2S":  EfficientNetV2S,
        "EfficientNetV2L":  EfficientNetV2L,
        "ConvNeXtBase":     ConvNeXtBase,
    }
    
    if model_name not in model_dict:
        raise ValueError(f"Model {model_name} is not supported.")

    # Load the base model without the top layers
    base_model = model_dict[model_name](weights='imagenet' if not save_weights else None,
                                        include_top=False,
                                        input_shape=(224, 224, 3))

    # Freeze all layers initially
    for layer in base_model.layers:
        layer.trainable = False

    # Calculate the index to start unfreezing layers
    from_index = int(np.round((len(base_model.layers) - 1) * (1.0 - trainable_layers / 100.0)))

    # Unfreeze layers from the calculated index onwards
    for layer in base_model.layers[from_index:]:
        layer.trainable = True

    # Add custom layers on top (Upper-Layers)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_value)(x)
    predictions = Dense(2, activation='softmax')(x)  # Assuming binary classification

    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Clear the base model from memory if needed (optional)
    del model_dict, base_model, from_index, x, predictions;    gc.collect()
    return model

In [ ]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

In [ ]:
def make_file():
    os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
    os.makedirs(LOGS_DIR, exist_ok=True)

In [ ]:
from keras import backend as K

def reset_memory():
    K.clear_session()
    gc.collect()

In [ ]:
PROJECT_PATH    = "/kaggle/working/"
CHECKPOINTS_DIR = os.path.join(PROJECT_PATH, "Checkpoints")
LOGS_DIR        = os.path.join(PROJECT_PATH, "Logs")
    
# Define the width you want for centering
WIDTH = 162

In [ ]:
def run_model(trainable_layers,
              optimizer_class,
              batch_size_value,
              dropout_value,
              model_name=None,
              save=False,
              epochs=5,
              patience=2):
    
    if model_name is None:
        models = ["ConvNeXtBase"]
        # models = ["ResNet50"]
    else:
        models = [model_name]  # Train only the specific model passed in

    best_score = float('-inf')  # Initialize best score to negative infinity
    best_model_info = None      # To store the best model information

    for model_name in models:
        # Clear any existing Keras session to free up memory
        tf.keras.backend.clear_session();    gc.collect()  # Optional: use garbage collection to ensure no memory leaks

        model = create_model(model_name, trainable_layers, dropout_value)  # Create the model
        optimizer_value = optimizer_class()  # Instantiate the optimizer

        model.compile(optimizer=optimizer_value,
                      loss='categorical_crossentropy',
                      metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])  # Compile the model

        trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
        print('')
        output = (f"Training model: {model_name}   "
                  f"Layers: {len(model.layers)}   "
                  f"Parameters: {model.count_params()}   "
                  f"Trainable parameters: {trainable_params}")
        print(output.center(WIDTH))
        print(f'#%% {"-" * 97} %'.center(WIDTH))

        # Prepare file paths if saving is enabled
        if save:
            solution = [trainable_layers, optimizer_value.__class__.__name__, batch_size_value, dropout_value]
            keyword = f"{model_name}-" + "-".join([str(el) for el in solution])
            checkpointPath = os.path.join(PROJECT_PATH, "Checkpoints", keyword) + ".weights.h5"
            csvLogPath = os.path.join(PROJECT_PATH, "Logs", keyword) + ".csv"
            make_file()  # Create directories

        # Set common callbacks
        callbacks = [
            TerminateOnNaN(),
            EarlyStopping(monitor="accuracy", mode="max", patience=patience)
        ]

        # Add saving-related callbacks if saving is enabled
        if save:
            callbacks += [
                ModelCheckpoint(checkpointPath,
                                save_best_only=True,
                                save_weights_only=True,
                                monitor="accuracy",
                                mode="max",
                                verbose=0),
                CSVLogger(csvLogPath, append=True)
            ]

        # Train the model
        history = model.fit(
            train_dataset,
            # validation_data=validation_dataset,
            batch_size=batch_size_value,
            epochs=epochs,
            callbacks=callbacks
        )
        
        if save:
            return model, history
        
        score = history.history['accuracy'][-1]
        # score = (val_acc + val_precision + val_recall) / 3  # Average score

        # Check if the current score is better than the best score
        if score > best_score:
            best_score = score  # Update best score
            best_model_info = {
                'model_name': model_name,
                'trainable_layers': trainable_layers,
                'optimizer': optimizer_value.__class__.__name__,
                'batch_size': batch_size_value,
                'dropout_value': dropout_value
            }

        # Clear the session after model is trained
        reset_memory()
        print(f"Model {model_name} cleared from memory".center(WIDTH))
        print()


    # return best_score, best_model_info  # Return best score and details of best model
        return model

In [ ]:
model, history = run_model(5,
                           Adam,
                           8,
                           0,
                           None,
                           True,
                           10,
                           # save=True,
                           patience=2)

In [ ]:
from tensorflow.keras.models import model_from_json, save_model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import shutil  # For creating zip files for download

# Save the model architecture to a JSON file
with open("model.json", "w") as json_file:
    json_file.write(model.to_json())


# To save the entire model (architecture and weights) to a single HDF5 file
model.save("entire_model.h5")

shutil.make_archive('entire_model', 'zip', '.', 'entire_model.h5')

In [ ]:
from IPython.display import FileLink
FileLink('entire_model.zip')

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file="model_architecture.png", show_shapes=True, show_layer_names=True)


In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
image_dir = '/kaggle/working/merged_images/0'
image_path = os.path.join(image_dir, os.listdir(image_dir)[0])

# Read the image
image = cv2.imread(image_path)

# Resize the image to the desired shape
image = cv2.resize(image, (224, 224))  # OpenCV uses (width, height) format for resizing

# Convert the image to a NumPy array and preprocess it for the model
image = np.array(image)
print(image.shape)

# Optionally, if your model expects the image in a specific format (e.g., scaled to [0, 1]),
# you may need to normalize it as follows:
image = image / 255.0  # Scale pixel values to [0, 1]

# Add a batch dimension (1, 224, 224, 3)
image = np.expand_dims(image, axis=0)

# Display the image (optional)
plt.imshow(image[0])  # Use image[0] to display the image without the batch dimension
plt.axis('off')  # Turn off axis labels
plt.show()

# Make predictions using the new model
result = model.predict(image)
print("Prediction Result:", result)

del image;    gc.collect()


In [ ]:
import os

# Specify the directory
working_dir = '/kaggle/working/'

# Iterate through files in the directory
for file_name in os.listdir(working_dir):
    # Check if the file is a .png file
    if file_name.endswith('.png'):
        # Construct full file path
        file_path = os.path.join(working_dir, file_name)
        # Remove the file
        os.remove(file_path)
        print(f"Deleted: {file_path}")

print("All .png images have been removed from the working directory.")

In [ ]:
epochs = range(len(history.history['accuracy']))
train_accuracy = history.history['accuracy']
train_precision = history.history['precision']  # Ensure these are recorded in history
train_recall = history.history['recall']
train_loss = history.history['loss']

# Accuracy Plot
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_accuracy, label='Train Accuracy', color='blue', marker='o')
plt.title('Training Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()

# Annotate training accuracy values
for i, acc in enumerate(train_accuracy):
    plt.text(i, acc + 0.01, f"{acc:.4f}", ha='center')

plt.tight_layout()
plt.savefig('training_accuracy.png')
plt.show()

# Precision Plot
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_precision, label='Train Precision', color='purple', marker='o')
plt.title('Training Precision Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.grid()

# Annotate training precision values
for i, prec in enumerate(train_precision):
    plt.text(i, prec + 0.01, f"{prec:.4f}", ha='center')

plt.tight_layout()
plt.savefig('training_precision.png')
plt.show()

# Recall Plot
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_recall, label='Train Recall', color='orange', marker='o')
plt.title('Training Recall Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.grid()

# Annotate training recall values
for i, rec in enumerate(train_recall):
    plt.text(i, rec + 0.01, f"{rec:.4f}", ha='center')

plt.tight_layout()
plt.savefig('training_recall.png')
plt.show()

# Loss Plot
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_loss, label='Train Loss', color='red', marker='o')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()

# Annotate training loss values
for i, loss in enumerate(train_loss):
    plt.text(i, loss + 0.01, f"{loss:.4f}", ha='center')

plt.tight_layout()
plt.savefig('training_loss.png')
plt.show()


In [ ]:
test_results = model.evaluate(test_dataset)
print(test_results)
test_loss = test_results[0]
test_accuracy = test_results[1]
test_precision = test_results[2]
test_recall = test_results[3]

# Test metrics for bar plot
test_metrics = ['Accuracy', 'Precision', 'Recall', 'Loss']
test_values = [test_accuracy, test_precision, test_recall, test_loss]

# Plot test metrics as bars
plt.figure(figsize=(8, 5))

# Bar plot with custom colors
bars = plt.bar(test_metrics, test_values, color=['#003049', '#219ebc', '#f77f00', '#d62828'])

# Set y-axis limits and labels
plt.ylim(0, 1)  # Adjust for metric ranges (may vary)
plt.title('Test Metrics', fontsize=14)
plt.ylabel('Value', fontsize=12)

# Annotate each bar with the corresponding value
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.02, f'{yval:.4f}', ha='center', va='bottom', fontsize=10)

# Tight layout for better spacing
plt.tight_layout()

# Save and display the plot
plt.savefig('test_metrics_bar_custom_colors.png')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get true labels and predicted labels for the test data
y_true = np.concatenate([y for x, y in test_dataset], axis=0).argmax(axis=1)
y_pred = np.concatenate([model.predict(x) for x, y in test_dataset], axis=0).argmax(axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.grid(False)
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.show()